## Load the IMDB dataset and create the vocabulary

In [1]:
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

EMBEDDING_DIM=50
VOCAB_SIZE=20000

# Get IMDB dataset
imdb = IMDB(split='train')

# Load English tokenizer, tagger, parser and NER
tokenizer = get_tokenizer('spacy', language='en')

# build the vocab
counter = Counter()
for i, (label, line) in enumerate(imdb):
    counter.update(tokenizer(line))

ordered_dict = OrderedDict(counter.most_common()[:VOCAB_SIZE])
vocab = vocab(ordered_dict)

# insert special tokens and set default index to 'unknown'
vocab.insert_token('<PAD>', 0)
vocab.insert_token('<UNK>', 1)
vocab.set_default_index(1)

100%|██████████| 84.1M/84.1M [00:03<00:00, 24.8MB/s]


## Create embedding vectors from GloVe

In [2]:
import torchtext as text

# load glove embeddings
vec = text.vocab.GloVe(name='6B', dim=50)
# create the embedding matrix, a torch tensor in the shape (num_words+1, embedding_dim)
word_emb = vec.get_vecs_by_tokens(vocab.get_itos())

.vector_cache/glove.6B.zip: 862MB [02:46, 5.17MB/s]                           
100%|█████████▉| 399999/400000 [00:14<00:00, 27182.81it/s]


## Build up train/test dataset

In [12]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# transform input text and label to ids
def process_text(text):
    return vocab(tokenizer(text))

label_to_ids = {'pos':0, 'neg':1}

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for (_label, _text) in batch:
        label_list.append(label_to_ids[_label])
        processed_text = torch.tensor(process_text(_text), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    # label must be in the same size as target
    label_list = torch.tensor(label_list, dtype=torch.float)[:,None]

    text_list = pad_sequence(text_list, batch_first=True)
    lengths = torch.tensor(lengths, dtype=torch.float)
    return label_list.to(device), text_list.to(device), lengths.to(device)

train_iter, test_iter = IMDB()

# transform datasets iterator into map style so that they can be repeatedly loaded in a new epoch
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

train_dataloader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=128,
                             shuffle=True, collate_fn=collate_batch)

In [17]:
# logistic model
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

class LogisticRegression(nn.Module):
    def __init__(self, word_vec, embed_dim):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        self.embedding = nn.Embedding(*(word_vec.size())).from_pretrained(word_vec, freeze=False)
        self.fc = nn.Linear(embed_dim, 1)
        self._init_weights()

    def _init_weights(self):
        """Initialize network parameters 
        """
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, lengths):
        embedded = self.embedding(text) # (batch_size, sent_len, emb_size)
        embedded = embedded.sum(dim = 1) / lengths[:, None] # (add one axis)
        return torch.sigmoid(self.fc(embedded))

class LSTMcustom(nn.Module):
    def __init__(self, word_vec, embed_dim):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        self.embedding = nn.Embedding(*(word_vec.size())).from_pretrained(word_vec, freeze=False)
        # Initialize LSTM model. The arguments are in this order input_dim, hidden_dim, n_layers
        
        # LSTM
        # 1 layer
        # self.lstm = nn.LSTM(embed_dim, 200, 1, bidirectional=True, batch_first = True)
        
        # 2 layers
        # self.lstm = nn.LSTM(embed_dim, 200, 2, bidirectional=True, batch_first = True)

        # to run the RNN, to save trouble, let's use back the same name for variable
        # 1 layer
        self.lstm = nn.RNN(embed_dim, 200, 1, bidirectional=True, batch_first=True)
        
        # Q1: What should be the input dimension of this linear layer? #ANSWER: 200
        self.fc = nn.Linear(200*2, 1)
        
        # Q2: set bidirectional = True in the LSTM and determine the input size of the FC layer
        # ANSWER: input size of the FC layer is 400
        
    def forward(self, text, lengths):
        embedded = self.embedding(text) # (batch_size, sent_len, emb_size)
        lstm_out,_ = self.lstm(embedded) # lstm_out is a 3d tensor (batch_size, sent_len, output_size). If you have a bidirectional LSTM, the outputsize will be 2*output_size
        
        # Q3: Select the hidden output of the last element in the sequence. Hint: Remember that you padded the sequence and you already know the length of the sequence.
        output_size = lstm_out.size()[2]
        batch_size = lstm_out.size()[0]

        c_length = lengths.cpu()

        last_indices = torch.FloatTensor(c_length - 1).long()

        last_indices_mod = last_indices.view(-1, 1, 1).expand(-1, 1, output_size).to(device)

        rows = torch.arange(0, batch_size).long()
        lstm_out = lstm_out[rows, last_indices, :]
          
        # Q4: Add multiple layers to the LSTM by passing argument num_layers=2,3,4 etc. to the LSTM function above. Test whether the performance improves. If not, explain what did you observe.
        # self.lstm = nn.LSTM(embed_dim, 200, 2, bidirectional=True, batch_first = True)
        # ANSWER:
        # For num_layer = 1, batch size = 128, accuracy = 63.91%
        # For num_layer = 1, batch size = 64, accuracy = 31.98%
        # For num_layer = 2, batch size = 64, accuracy = 31.98%
        # Due to limited CUDA memory, the batch size is lowered to 64. There is a huge decrease 
        # in terms of accuracy, most likely is caused by change of batch size
        # However, number of layers doesnt affect the test accuracy judging from num_layer = 2 and 1
        # yield the same accuracy for the same batch size
        # Due to time constraint(took me 1 hr to run on num_layer=2), I wasnt able to run 
        # num_layer = 3 and 4. 
        
        # Q5: Replace the LSTM layer with simple RNN layer and compare the performance between LSTM and RNN.
        # rnn_out, _ = self.rnn(embedded)
        # return torch.sigmoid(self.fc(rnn_out)) 
        
        # ANSWER: 
        # For num_layer = 1, batch size = 128, accuracy = 63.91%
        # As shown, the accuracy for RNN with the above parameters give the same result compared to LSTM.
        # Although more combination of number of layers and batch size should be run, but we can infer 
        # that for simple RNN layer, both models will have similar or even same performance, and both relies
        # on batch size.

        return torch.sigmoid(self.fc(lstm_out))

In [18]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, lengths) in enumerate(dataloader):
        optimizer.zero_grad()
        # forward propagation
        predicted_label = model(text, lengths)
        # calculate loss and backpropagate to model paramters
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # update parameters by stepping the optimizer
        optimizer.step()
        total_acc += ((predicted_label > 0.5) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, lengths) in enumerate(dataloader):
            predicted_label = model(text, lengths)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [19]:
# Hyperparameters
EPOCHS = 10 # epoch

model = LSTMcustom(word_vec=word_emb, embed_dim=EMBEDDING_DIM).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()
total_accu = None


for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)

| epoch   1 |    50/  196 batches | accuracy    0.527
| epoch   1 |   100/  196 batches | accuracy    0.533
| epoch   1 |   150/  196 batches | accuracy    0.523
| epoch   2 |    50/  196 batches | accuracy    0.521
| epoch   2 |   100/  196 batches | accuracy    0.523
| epoch   2 |   150/  196 batches | accuracy    0.542
| epoch   3 |    50/  196 batches | accuracy    0.534
| epoch   3 |   100/  196 batches | accuracy    0.533
| epoch   3 |   150/  196 batches | accuracy    0.526
| epoch   4 |    50/  196 batches | accuracy    0.534
| epoch   4 |   100/  196 batches | accuracy    0.533
| epoch   4 |   150/  196 batches | accuracy    0.550
| epoch   5 |    50/  196 batches | accuracy    0.531
| epoch   5 |   100/  196 batches | accuracy    0.542
| epoch   5 |   150/  196 batches | accuracy    0.528
| epoch   6 |    50/  196 batches | accuracy    0.539
| epoch   6 |   100/  196 batches | accuracy    0.534
| epoch   6 |   150/  196 batches | accuracy    0.536
| epoch   7 |    50/  196 ba

In [20]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.2f}%'.format(accu_test))

test accuracy    63.91%
